In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya

In [ ]:
using Printf

In [ ]:
import DataFrames: DataFrame, rename!

In [ ]:
using OrderedCollections

In [ ]:
models_dir = "/astro/dboyea/dwarfs/analysis/sculptor"

In [ ]:
sims_dir = "/astro/dboyea/dwarfs/simulations/sculptor"

In [ ]:
function load_profile(name) 
    path = joinpath(models_dir, "$name/profiles.hdf5")
    profiles = LilGuys.read_structs_from_hdf5(path, LilGuys.MassProfile3D)
    idx = parse.(Int, first.(profiles))
    return profiles[sortperm(idx)]
    
end

In [ ]:
t_max_host = 316 # computed with agama at peri=50.95 
M_max_host = 52.437
r_peri = 50.95
ρ_peri = M_max_host / (4π/3 * r_peri^3)

# Plotting functions

In [ ]:
log_r_label = "log r / kpc"
log_rho_label = L"\log \rho"

In [ ]:
function compare_densities(profiles)
    fig = Figure()
    ax = Axis(fig[1,1],
        xlabel=log_r_label,
        ylabel = log_rho_label,
        )
    
    ax.limits=(-2, 3, -12, 0)


    for i in eachindex(profiles)
        label, profs = profiles[i]
        profile = profs[1].second
        lines!(profile.log_r, log10.(profile.rho),
            linestyle=:dot,
            color=COLORS[i]
        )
    
        profile = profs[end].second
        lines!(profile.log_r, log10.(profile.rho), 
            color=COLORS[i],
            label=label
        )    
    end
    
    axislegend()

    fig
end

In [ ]:
function compare_vcirc(profiles; errskip=1, legend=true)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=log_r_label,
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        lines!(log10.(x), log10.(y*V2KMS), label=label)
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        scatter!(log10.(x[end]), log10.(y[end]*V2KMS))
    end
    if legend
        axislegend(position=:lt)
    end
    fig
end

In [ ]:
function compare_vcirc_time(profiles; errskip=1, legend=true)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel="time / Gyr",
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.time * T2GYR for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        lines!((x), log10.(y*V2KMS), label=label)
    end
    
    if legend
        axislegend(position=:lt)
    end
    fig
end

In [ ]:
function compare_tcirc(profiles; errskip=1)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel="time / Gyr",
        ylabel=L"$\log\, t_\textrm{circ}$ / Gyr",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [parse(Float64, i) for (i, prof) in profs]
        y = [prof.r_circ_max / prof.v_circ_max for (_, prof) in profs]
        
        lines!(x, log10.(y*T2GYR), label=label)
    end
   
    fig
end

# Particle Number

In [ ]:
profiles = [
    "smallperi" => load_profile("1e7_V31_r3.2/orbit_smallperi"),
    "mean" => load_profile("1e7_V31_r3.2/orbit_mean"),
    "V+21 (1e6)" => load_profile("1e6_V31_r3.2/vasiliev+21_mean"),
    #"heavy" => load_profile("1e6_V40_r5.9/orbit_mean"),
    ];

In [ ]:
fig = compare_vcirc(profiles, legend=false)
prof = profiles[1].second[1].second

r_circ_tidal, v_circ_tidal = LilGuys.EN21_tidal_track(prof.r_circ_max, prof.v_circ_max, x_min=0.1)


lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dot, label="initial $(profiles[1].first)")
prof = profiles[1].second[end].second

lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dash, label="final $(profiles[1].first)")

lines!(log10.(r_circ_tidal), log10.(v_circ_tidal * V2KMS), label = "EN21 tidal track", color=:black, alpha=0.3)


ax = fig.content[1]
ax.limits = (-1, 2, 1, 1.5)
ax.xlabel = L"\log\,(r_\textrm{circ}\,/\,\textrm{kpc})"

axislegend(position=:rb)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel="time / Gyr",
    ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
    )
pi = 1


for i in eachindex(profiles)
    label, profs = profiles[i]
    x = [prof.time * T2GYR for (_, prof) in profs]
    println(x)
    y = [prof.v_circ_max for (_, prof) in profs]
    lines!(log10.(x), log10.(y*V2KMS), label=label)
end

fig

In [ ]:
profiles = [
    "smallperi (1e6)" => load_profile("1e6_V31_r3.2/orbit_smallperi"),
    "midsmallperi (1e6)" => load_profile("1e6_V31_r3.2/orbit_midsmallperi"),
    "lmc" => load_profile("1e6_V31_r3.2/vasiliev+21_mean"),
    #"heavy" => load_profile("1e6_V40_r5.9/orbit_mean"),
    ];

In [ ]:
fig = compare_vcirc(profiles, legend=false)
prof = profiles[end].second[1].second

r_circ_tidal, v_circ_tidal = LilGuys.EN21_tidal_track(prof.r_circ_max, prof.v_circ_max, x_min=0.1)
lines!(log10.(r_circ_tidal), log10.(v_circ_tidal * V2KMS), label = "EN21 tidal track", color=:black, alpha=0.3)


lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dot, label="initial (smallperi)")
prof = profiles[end].second[end].second

lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dash, label="final (smallperi)")



ax = fig.content[1]
ax.limits = (-1, 2, 1, 1.5)
ax.xlabel = L"\log\,(r_\textrm{circ}\,/\,\textrm{kpc})"

axislegend(position=:rb)
fig

### Changing the orbits

In [ ]:
profiles = [
    "mean" => load_profile("1e6_V31_r3.2/orbit_mean"),
    "smallperi" => load_profile("1e6_V31_r3.2/orbit_smallperi"),
    "v+21" => load_profile("1e6_V31_r3.2/vasiliev+21_mean"),
    "v+21 smallperi" => load_profile("1e6_V31_r3.2/vasiliev+21_smallperi"),
    "v+21 smallperi + rmax=4.2" => load_profile("1e6_V31_r4.2/vasiliev+21_smallperi"),
    #"heavy" => load_profile("1e6_V40_r5.9/orbit_mean"),
    ];

In [ ]:
fig = compare_vcirc(profiles, legend=false)
prof = profiles[end].second[1].second

r_circ_tidal, v_circ_tidal = LilGuys.EN21_tidal_track(prof.r_circ_max, prof.v_circ_max, x_min=0.1)
lines!(log10.(r_circ_tidal), log10.(v_circ_tidal * V2KMS), label = "EN21 tidal track", color=:black, alpha=0.3)


lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dot, label="initial (smallperi)")
prof = profiles[end].second[end].second

lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dash, label="final (smallperi)")



ax = fig.content[1]
ax.limits = (-1, 2, 1, 1.5)
ax.xlabel = L"\log\,(r_\textrm{circ}\,/\,\textrm{kpc})"

axislegend(position=:rb)
fig

In [ ]:
profiles = OrderedDict(
    "mean" => load_profile("1e6_V31_r3.2/orbit_mean"),
    #"smallperi" => load_profile("1e6_V31_r3.2/orbit_smallperi"),
    "v+21" => load_profile("1e6_V31_r3.2/vasiliev+21_mean"),
    #"v+21 smallperi" => load_profile("1e6_V31_r3.2/vasiliev+21_smallperi"),
    "v+21 heavylmc" => load_profile("1e6_V31_r3.2/vasiliev+21_heavylmc"),
    "v+21 smallperi heavylmc" => load_profile("1e6_V31_r3.2/vasiliev24_L3M11_smallperi"),
    #"heavy" => load_profile("1e6_V40_r5.9/orbit_mean"),
    );

In [ ]:
fig = compare_vcirc(profiles, legend=false)
prof = profiles[end].second[1].second

r_circ_tidal, v_circ_tidal = LilGuys.EN21_tidal_track(prof.r_circ_max, prof.v_circ_max, x_min=0.1)
lines!(log10.(r_circ_tidal), log10.(v_circ_tidal * V2KMS), label = "EN21 tidal track", color=:black, alpha=0.3)


lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dot, label="initial (smallperi)")
prof = profiles[end].second[end].second

lines!(log10.(prof.r_circ), log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dash, label="final (smallperi)")



ax = fig.content[1]
ax.limits = (-1, 2, 1, 1.5)
ax.xlabel = L"\log\,(r_\textrm{circ}\,/\,\textrm{kpc})"

axislegend(position=:rb)
fig

In [ ]:
compare_vcirc_time(profiles)

In [ ]:
ps = profiles["v+21 smallperi heavylmc"]

In [ ]:
profs = load_profile("1e6_V31_r3.2/vasiliev24_L2M11_extremeperi")

In [ ]:
compare_vcirc_time(["1"=>prof])

In [ ]:
 compare_vcirc(["1" => prof])

In [ ]:
profi = prof[1].second
r = profi.r_circ
t = r ./ profi.v_circ
N = profi.n_circ
h = 0.024
log_Λ = log.(r/h)

t_relax = @. t * N / (8*log_Λ)


In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel="log r / kpc",
    ylabel = "log t relax / Gyr",
    limits=(-2, 2, -3, 2)
    )


lines!(log10.(r), log10.(t_relax*T2GYR))
lines!(log10.(r), log10.(t*T2GYR))

vlines!(-0.3)
fig

## L3M11

In [ ]:
profs = load_profile("1e6_V31_r3.2/vasiliev24_L3M11_extremeperi")

In [ ]:
compare_vcirc_time(["maximum"=>profs])

In [ ]:
    
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel=log_r_label,
    ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
    )
pi = 1


x = [prof.r_circ_max for (_, prof) in profs]
y = [prof.v_circ_max for (_, prof) in profs]
scatterlines!(log10.(x), log10.(y*V2KMS),)

fig

In [ ]:
[prof.time for (_, prof) in profs] * T2GYR

In [ ]:
 compare_vcirc(["1" => profs])

In [ ]:
profi = profs[1].second
r = profi.r_circ
t = r ./ profi.v_circ
N = profi.n_circ
h = 0.024
log_Λ = log.(r/h)

t_relax = @. t * N / (8*log_Λ)


In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel="log r / kpc",
    ylabel = "log t relax / Gyr",
    limits=(-2, 2, -3, 2)
    )


lines!(log10.(r), log10.(t_relax*T2GYR))
lines!(log10.(r), log10.(t*T2GYR))

vlines!(-0.3)
fig